In [2]:
from pathlib import Path
import hashlib
import pickle
import logging
import os
import shutil
import sys

In [3]:
source_path = '/Volumes/320 GB/' #Specify the source location here
destination_path = '/Users/georgyjose/Documents/backup_photos/' #Specify the target location here
existing_hashes = 'hashes.pickle'
existing_logs = 'errors.pickle'

In [4]:
def hash_file(filepath):
    with open(filepath, 'rb') as f:
        return hashlib.md5(f.read()).hexdigest()

In [5]:
def clear_hashes():
    pickle_out = open(existing_hashes,"wb")
    pickle.dump([], pickle_out)
    pickle_out.close()

In [6]:
def read_hashes():
    pickle_in = open(existing_hashes,"rb")
    hash_list = pickle.load(pickle_in)
    return hash_list

In [7]:
def write_hashes(hash_list):
    pickle_out = open(existing_hashes,"wb")
    pickle.dump(hash_list, pickle_out)
    pickle_out.close()

In [8]:
def is_duplicate(image):
    existing_hashes = read_hashes()
    hash_of_image = hash_file(image)
    if hash_of_image in existing_hashes:
        return True
    else:
        existing_hashes.append(hash_of_image)
        write_hashes(existing_hashes)
        return False

In [9]:
img_file_formats = ['.jpx',
 '.jpg',
 '.gif',
 '.jfif',
 '.j2k',
 '.jif',
 '.j2c',
 '.jp2',
 '.fpx',
 '.png',
 '.jpeg',
 '.tif',
 '.pcd',
 '.tiff']

In [10]:
# clear_hashes()
logging.basicConfig(filename='app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

In [11]:
files_copied = 0
duplicate_files = 0
for root, dirs, files in os.walk(source_path):
    path_from_current_root = root.split(source_path, 1).pop()
    create_folder = os.path.join(destination_path,path_from_current_root)
    os.makedirs(create_folder, exist_ok = True)
    for file in files:
        try:
            file_path = os.path.join(root,file)
            if list(filter(file.lower().endswith, img_file_formats)) != []:
                files_is_duplicate = is_duplicate(os.path.join(root,file))
                if not files_is_duplicate:
                    shutil.copy(file_path, os.path.join(destination_path, path_from_current_root))
                    files_copied += 1
                    sys.stdout.write('\r')
                    sys.stdout.write(str(files_copied) + " files copied.")
                    sys.stdout.flush()
                else:
                    duplicate_files += 1
        except:
            logging.error(str(os.path.join(root, file)))
            
            
                
print(str(files_copied) + " image files copied in total.")
print(str(duplicate_files) + " duplicate files were ignored.")

76041 files copied.76041 image files copied in total.
157857 duplicate files were ignored.
